In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, r2_score, mean_squared_error

In [7]:
df = pd.read_parquet('..\..\ETH-Full-1H.parquet') # 2 dir up

In [8]:
df

,Open,High,Low,Close,Volume,Trades
Datetime,,,,,,
2017-09-01 00:00:00,384.79,388.74,384.79,388.74,162.97779,131
2017-09-01 01:00:00,388.69,394.06,387.53,391.37,248.72949,262
2017-09-01 02:00:00,392.88,394.39,389.46,390.88,198.82856,225
2017-09-01 03:00:00,391.70,393.13,390.86,390.86,141.56114,138
2017-09-01 04:00:00,392.65,394.27,387.64,390.33,135.65894,128
...,...,...,...,...,...,...
2023-04-23 19:00:00,1846.76,1852.35,1844.92,1846.49,16550.18290,21648
2023-04-23 20:00:00,1846.49,1852.30,1838.03,1848.89,13113.47550,19445
2023-04-23 21:00:00,1848.88,1859.32,1847.59,1854.66,7855.61330,13824


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 49337 entries, 2017-09-01 00:00:00 to 2023-04-23 23:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    49337 non-null  float64
 1   High    49337 non-null  float64
 2   Low     49337 non-null  float64
 3   Close   49337 non-null  float64
 4   Volume  49337 non-null  float64
 5   Trades  49337 non-null  int32  
dtypes: float64(5), int32(1)
memory usage: 2.4 MB


In [10]:
df.describe()

,Open,High,Low,Close,Volume,Trades
count,49337.000000,49337.000000,49337.000000,49337.000000,49337.000000,49337.000000
mean,1129.787828,1137.506014,1121.512988,1129.812692,24069.305450,23040.347123
std,1152.173610,1159.468381,1144.315048,1152.175379,28507.816928,29913.374971
min,82.160000,82.950000,81.790000,82.170000,1.459450,7.000000
25%,221.110000,222.660000,219.630000,221.110000,7683.970060,4858.000000
50%,538.930000,543.230000,534.400000,538.870000,15761.451700,12221.000000
75%,1732.550000,1744.530000,1719.620000,1732.560000,30009.152030,30603.000000
max,4846.940000,4868.000000,4833.190000,4846.710000,493227.882820,536376.000000


В качестве наивного прогноза используется закрытие предыдущего периода

In [11]:
df['Close_prediction'] = df['Close'].shift(1)

In [12]:
# Первое значение пустое, удаляю всю строку
df.dropna(inplace=True)

In [13]:
# Выделяю вектор правильных ответов и вектор наивного предсказагния
y_true = df['Close']
y_pred = df['Close_prediction']

## Metrics

### SSE сумма квадратов ошибок

In [14]:
(y_true - y_pred).dot(y_true - y_pred)

12362684.549799995

### MSE средняя квадратичная ошибка

In [15]:
mean_squared_error(y_true, y_pred)

250.58141214934324

In [16]:
# расчет вручную
(y_true - y_pred).dot(y_true - y_pred) / len(y_true)

250.58141214934318

### RMSE

In [17]:
mean_squared_error(y_true, y_pred, squared=False)

15.829763490000198

In [18]:
# расчет вручную
np.sqrt((y_true - y_pred).dot(y_true - y_pred) / len(y_true))

15.829763490000197

### R^2 score. Инвариантная метрика.
Слишком высокое значение что бы быть правдой

In [19]:
r2_score(y_true, y_pred)

0.999811237676953

### MAPE средняя абсолютная процентная ошибка. Инвариантная ошибка.

In [20]:
mean_absolute_percentage_error(y_true, y_pred)

0.006642962280889733

### sMAPE Симметричная средняя абсолютная процентная ошибка

In [21]:
def smape(y_true, y_pred):
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = numerator / denominator
    return ratio.mean()

In [22]:
smape(y_true, y_pred)

0.006639043992486268